In [1]:
import pandas as pd
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
submission_df = pd.read_csv('../input/sample_submission.csv')

In [2]:
train_df

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [3]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

%matplotlib inline
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [4]:
def SMAPE (forecast, actual):
    """Returns the Symmetric Mean Absolute Percentage Error between two Series"""
    masked_arr = ~((forecast==0)&(actual==0))
    diff = abs(forecast[masked_arr] - actual[masked_arr])
    avg = (abs(forecast[masked_arr]) + abs(actual[masked_arr]))/2
    
    print('SMAPE Error Score: ' + str(round(sum(diff/avg)/len(forecast) * 100, 2)) + ' %')

In [5]:
def series_to_supervised(data, window=1, lag=1, dropnan=True, exogenous = ['item', 'store'], value = ['sales']):
    cols= list()
    names = exogenous.copy()
    cols.append(data[exogenous])
    
    for i in range(window, 0, -1):
        cols.append(data[value].shift(i))
        names += [('sales(t-%d)' % (i))]

    cols.append(data[value])
    names += ['sales(t)']

    cols.append(data[value].shift(-lag))
    names += [('sales(t+%d)' % ( lag))]
    
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values if needed
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [6]:
sample = train_df[(train_df.item == 1)&(train_df.store==1)]
sample.sample()

,date,store,item,sales
805,2015-03-17,1,1,15


In [29]:
window = 355
lag_size = 90
series = series_to_supervised(sample.drop('date', axis=1), window=window, lag=lag_size)
series

,item,store,sales(t-355),sales(t-354),sales(t-353),sales(t-352),sales(t-351),sales(t-350),sales(t-349),sales(t-348),...,sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t),sales(t+90)
355,1,1,13.0,11.0,14.0,13.0,10.0,12.0,10.0,9.0,...,16.0,15.0,15.0,8.0,18.0,7.0,13.0,11.0,9,20.0
356,1,1,11.0,14.0,13.0,10.0,12.0,10.0,9.0,12.0,...,15.0,15.0,8.0,18.0,7.0,13.0,11.0,9.0,8,25.0
357,1,1,14.0,13.0,10.0,12.0,10.0,9.0,12.0,9.0,...,15.0,8.0,18.0,7.0,13.0,11.0,9.0,8.0,17,4.0
358,1,1,13.0,10.0,12.0,10.0,9.0,12.0,9.0,9.0,...,8.0,18.0,7.0,13.0,11.0,9.0,8.0,17.0,6,15.0
359,1,1,10.0,12.0,10.0,9.0,12.0,9.0,9.0,7.0,...,18.0,7.0,13.0,11.0,9.0,8.0,17.0,6.0,16,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,1,1,24.0,29.0,18.0,18.0,16.0,16.0,32.0,32.0,...,20.0,20.0,28.0,37.0,24.0,14.0,18.0,27.0,23,14.0
1732,1,1,29.0,18.0,18.0,16.0,16.0,32.0,32.0,26.0,...,20.0,28.0,37.0,24.0,14.0,18.0,27.0,23.0,28,19.0
1733,1,1,18.0,18.0,16.0,16.0,32.0,32.0,26.0,20.0,...,28.0,37.0,24.0,14.0,18.0,27.0,23.0,28.0,30,15.0
1734,1,1,18.0,16.0,16.0,32.0,32.0,26.0,20.0,21.0,...,37.0,24.0,14.0,18.0,27.0,23.0,28.0,30.0,21,27.0


In [30]:
labels = series.iloc[:, -1]
series = series.iloc[:, :-1]%recall

NameError: name 'recall' is not defined

In [31]:
X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.2, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (1104, 359)
Validation set shape (277, 359)


,item,store,sales(t-355),sales(t-354),sales(t-353),sales(t-352),sales(t-351),sales(t-350),sales(t-349),sales(t-348),...,sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t),sales(t+90)
1037,1,1,25.0,24.0,18.0,17.0,18.0,19.0,17.0,28.0,...,18.0,8.0,17.0,19.0,30.0,26.0,16.0,25.0,26,16.0
1273,1,1,21.0,33.0,26.0,30.0,19.0,22.0,19.0,20.0,...,28.0,15.0,25.0,25.0,23.0,31.0,31.0,32.0,26,27.0
1458,1,1,16.0,26.0,7.0,13.0,9.0,18.0,13.0,8.0,...,12.0,20.0,21.0,13.0,20.0,16.0,10.0,16.0,21,12.0
1299,1,1,20.0,26.0,20.0,25.0,22.0,19.0,33.0,14.0,...,33.0,34.0,37.0,24.0,25.0,22.0,34.0,26.0,34,25.0
1292,1,1,13.0,27.0,26.0,22.0,29.0,28.0,19.0,20.0,...,34.0,35.0,28.0,18.0,27.0,30.0,22.0,33.0,34,32.0


In [21]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Train set shape (1104, 359, 1)
Validation set shape (277, 359, 1)


In [11]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [48]:
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(100, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss='mse', optimizer=adam)
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 357, 64)           256       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 178, 64)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 11392)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               1139300   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 1,139,657
Trainable params: 1,139,657
Non-trainable params: 0
_________________________________________________________________


In [49]:
%%time
cnn_history = model_cnn.fit(X_train_series, Y_train, validation_data=(X_valid_series, Y_valid), epochs=epochs, verbose=2)

Train on 1104 samples, validate on 277 samples
Epoch 1/40
 - 0s - loss: 301.0077 - val_loss: 39.9655
Epoch 2/40
 - 0s - loss: 39.3413 - val_loss: 34.7818
Epoch 3/40
 - 0s - loss: 35.3353 - val_loss: 31.7000
Epoch 4/40
 - 0s - loss: 32.7497 - val_loss: 29.8310
Epoch 5/40
 - 0s - loss: 30.4546 - val_loss: 27.8524
Epoch 6/40
 - 0s - loss: 28.4941 - val_loss: 25.8046
Epoch 7/40
 - 0s - loss: 26.7067 - val_loss: 24.9026
Epoch 8/40
 - 0s - loss: 25.6657 - val_loss: 23.9585
Epoch 9/40
 - 0s - loss: 24.5734 - val_loss: 24.6998
Epoch 10/40
 - 0s - loss: 24.2942 - val_loss: 23.2430
Epoch 11/40
 - 0s - loss: 23.6825 - val_loss: 23.0026
Epoch 12/40
 - 0s - loss: 23.1498 - val_loss: 22.8825
Epoch 13/40
 - 0s - loss: 22.8299 - val_loss: 23.1733
Epoch 14/40
 - 0s - loss: 22.3455 - val_loss: 22.7957
Epoch 15/40
 - 0s - loss: 22.1221 - val_loss: 22.6398
Epoch 16/40
 - 0s - loss: 21.8037 - val_loss: 22.6829
Epoch 17/40
 - 0s - loss: 21.7821 - val_loss: 22.6726
Epoch 18/40
 - 0s - loss: 21.9271 - val_los

In [72]:
MPL_prediction = model_cnn.predict(np.array(series).reshape(series.shape[0],series.shape[1],1)[-90:]).flatten()
SMAPE(MPL_prediction, labels[-90:])

SMAPE Error Score: 17.4 %


,item,store,sales(t-355),sales(t-354),sales(t-353),sales(t-352),sales(t-351),sales(t-350),sales(t-349),sales(t-348),...,sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t),sales(t+90)


In [ ]:
SMAPE(check, result)